In [2]:
#importing the relevant libraries
import requests
import lxml.html as lh
import pandas as pd

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [5]:
#extracting the table from the data frame 
df=pd.read_html(url)[0]

In [6]:
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


Wrangling the data 

In [7]:
#the first step of dropping the not assigned boroughs
import numpy as np
df.replace("Not assigned", np.nan, inplace=True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [8]:
#dropping all the boroughs that are not assigned
df.dropna(subset=["Borough"],axis =0, inplace = True)
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Combining the neighourhoods with the same postal code

In [9]:
df =df.groupby(["Postal Code","Borough"])
df=df["Neighbourhood"].apply(', '.join).reset_index() 
df.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Changing all the not assigned neighbourhood values to the borough

In [10]:
for ii in range(len(df)):
    if df['Neighbourhood'].iloc[ii] =='Not assigned':
        df['Neighbourhood'].iloc[ii]= df['Borough'].iloc[ii]

In [11]:
df.shape

(103, 3)